# Reveal Cancer Invasion Strategy with Reinforcement Learning
We demonstrate how cancer cells learn to reproduce and invade using reinforcement learning. The idea originated from a proposal to Cell Fate Symposium 2018 (see proposal [here](https://www.dropbox.com/s/er54z58cbda2bkn/cell%20fate%20full%20proposal.pdf?dl=0)).

At the beginning of each episode, one cancer cell at the center of a fixed square domain with even-distributed nutrients is initialized. We imagine that an agent is playing a game in which it takes cues from the microenvironment of a cancer cell and instructs the cell whether to reproduce or migrate. The cost of nutrient to reproduce is higher than moving around. The nutrient is limited and diffuse within the domain with a no-flux boundary condition. When the nutrient is below certain threshold the cells start dying. The goal is to maximize the number of offsprings that successfully exit the domain. For this goal, we design rewarding as follows:
* reproduction results in a positive reward and
* death leads to a negative reward of the same amount.
* extra reward is given when a cancer cell is successfully exit the domain.

Here we employed Policy Gradient algorithm to learn possible strategy of cancer invasion.  Policy function takes input of ambient nutrient level and stochasticaly maps to the action that can be taken by the cancer cell: moving to one of the nearby sites or reproduce. Here policy function is approximated as a neutral net with one hidden layer. Run the "cell" below to see how the cancer cells improve its reward with training with more and more episode. The bottom graph is the running average reward over the episodes. The two grid plots show details of several typical episodes. Note salient behavior changes after 100 episode the cancer cells compare to the 1st episode.

This is a simple sketch of the idea. In the future, we hope to reveal interesting morphological patterns of cancer invasion by adding more hidden layers and more biologically meaningfully features to the current model.

In [ ]:
%matplotlib nbagg
""" Trains an agent with (stochastic) Policy Gradients on Cancer Invasion. """
import numpy as np
import pickle
from cancer_env import CancerEnv

# hyperparameters
H = 400 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 3 * 3 # input dimensionality: 3x3 grid nut
if resume:
  model = pickle.load(open('save.p', 'rb'))
else:
  model = {}
  model['W1'] = 0.1*np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
  model['W2'] = 0.1*np.random.randn(H,9) / np.sqrt(H) # 9 actions
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory


def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = model['W1'] @ x
  h[h<0] = 0 # ReLU nonlinearity
  logp = h @ model['W2']
  p = np.exp(logp) / np.exp(logp).sum()
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = eph.T @ epdlogp
  #dh = np.outer(epdlogp, model['W2'])
  dh = epdlogp @ model['W2'].T
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}


env = CancerEnv(19)
observation = env.reset()  # 3 by 3 neibnut
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
istep = 0
while True:
  istep += 1
  if render and istep % 2 == 0:
    env.render()
    istep = 0
  # preprocess the observation, set input to network to be difference image
  x = observation.ravel()

  # forward the policy network and sample an action from the returned probability
  aprob, h = policy_forward(x)
  action = np.random.choice(range(9), p=aprob)  # roll the dice!

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state

  y = np.zeros(9)
  y[action] = 1
  dlogps.append(y - aprob) # !grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
  observation, reward, done = env.pg_step(action)
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    #print(('ep %d: game finished, total reward: %f' % (episode_number, reward_sum)) + ('' if reward_sum < 11 else '!!!!'))
    episode_number += 1
    if episode_number in [1, 50, 100, 1000, 1500]:
      render = True
    else:
      render = False
    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)  # int????
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr = discounted_epr - np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    epdlogp *= discounted_epr # !modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.items():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print('ep %d : resetting env. episode reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
    env.plot_extra(episode_number, running_reward)
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env